In [1]:
import pandas as pd

import tensorflow as tf


In [2]:
CSV_COLUMNS_NAMES = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Versicolor','Virginica']

In [3]:
train_path = tf.keras.utils.get_file('iris_training.csv','http://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv')

test_path =  tf.keras.utils.get_file('iris_test.csv','http://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv')

train = pd.read_csv(train_path,names= CSV_COLUMNS_NAMES,header=0)

test = pd.read_csv(test_path,names=CSV_COLUMNS_NAMES,header=0)

train.head()

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [4]:
train_y = train.pop('Species')

test_y = test.pop('Species')



In [5]:
def input_fn(features,labels,training = True,batch_size = 256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)  


In [6]:
my_feature_columns = []

for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key = key))

print(my_feature_columns)   

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [7]:
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,hidden_units=[30,10],n_classes=3)

classifier.train(input_fn=lambda:input_fn(train,train_y,training=True),steps = 5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmphq8ki4bs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initiali

In [8]:
eval_result =  classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))

print('Test Set  Accuracy: {accuracy:0.3f}'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-06-15T05:50:16
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmphq8ki4bs/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.62546s
INFO:tensorflow:Finished evaluation at 2022-06-15-05:50:17
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.6, average_loss = 0.5439561, global_step = 5000, loss = 0.5439561
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmphq8ki4bs/model.ckpt-5000
Test Set  Accuracy: 0.600


In [9]:
7def input_fn(features,batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict = {}

print('Please Type numeric Values as prompted ')

for feature in features:
  valid = True
  while valid:
    val = input(feature + ': ')
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda : input_fn(predict) )

for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}"({:.1f}%)'.format(SPECIES[class_id],100*probability))

Please Type numeric Values as prompted 
SepalLength: 3
SepalLength: 3
SepalLength: 5
SepalLength: 4
SepalLength: 2
SepalLength: 4
SepalLength: 7
SepalLength: 4
SepalLength: 1
SepalLength: 6
SepalLength: 3
SepalLength: 5
SepalLength: 


ValueError: ignored